# Multimodal Hate Detection via Zero/Few-Shot VLMs

This notebook provides an interactive environment for evaluating memes using the hate detection pipeline.

## Pipeline
```
Image → VLM → Image description + OCR → LLM → HATE/NON-HATE
```

## Setup

In [ ]:
# Install dependencies if needed
# !pip install -r ../requirements.txt

In [ ]:
import sys
import asyncio
from pathlib import Path
from IPython.display import display, Image, Markdown

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src.config import settings
from src.pipeline import HateDetector
from src.evaluation import MetricsCalculator, FailureModeAnalyzer

In [ ]:
# Initialize the detector
detector = HateDetector()

# Check health
health = await detector.health_check()
print(f"VLM: {'✅' if health['vlm'] else '❌'} | LLM: {'✅' if health['llm'] else '❌'}")

## Single Meme Analysis

In [ ]:
# Helper function to display results nicely
def display_result(result, show_image=True):
    if show_image and Path(result.image_path).exists():
        display(Image(filename=result.image_path, width=400))
    
    if result.error:
        print(f"❌ Error: {result.error}")
        return
    
    print("\n📸 VLM Analysis:")
    print(f"   Visual: {result.vlm_output.visual_description[:200]}...")
    print(f"   OCR: {result.vlm_output.ocr_text}")
    print(f"   Meaning: {result.vlm_output.implicit_meaning[:200]}...")
    print(f"   Target: {result.vlm_output.target_group}")
    
    label_emoji = "🔴" if result.classification.label == "HATE" else "🟢"
    print(f"\n{label_emoji} Result: {result.classification.label}")
    print(f"   Justification: {result.classification.justification}")
    print(f"   Mode: {result.inference_mode} | Time: {result.processing_time:.2f}s")

In [ ]:
# Analyze a single meme
image_path = "../data/samples/your_meme.jpg"  # Change this to your meme path

# Choose mode: "zero_shot", "few_shot", or "cot"
mode = "zero_shot"

result = await detector.detect(image_path, mode)
display_result(result)

## Compare All Inference Modes

In [ ]:
# Compare all three modes on the same meme
image_path = "../data/samples/your_meme.jpg"  # Change this

results = await detector.compare_modes(image_path)

print("Mode Comparison:")
print("-" * 60)
for mode, result in results.items():
    label = result.classification.label
    emoji = "🔴" if label == "HATE" else "🟢"
    print(f"{mode:12} | {emoji} {label:10} | {result.processing_time:.2f}s")
    print(f"             | {result.classification.justification[:50]}...")
    print("-" * 60)

## Batch Evaluation

In [ ]:
# Evaluate multiple memes
meme_paths = [
    "../data/samples/meme1.jpg",
    "../data/samples/meme2.jpg",
    "../data/samples/meme3.jpg",
]

batch_result = await detector.detect_batch(meme_paths, mode="zero_shot")
print(batch_result.accuracy_summary)

## Dataset Evaluation

In [ ]:
from data.datasets import MultiBullyLoader, BanglaLoader

# Load dataset
dataset_path = "../data/datasets/multibully"
loader = MultiBullyLoader(dataset_path)

# Get statistics
stats = loader.get_statistics()
print(f"Dataset: {loader.name}")
print(f"Total samples: {stats['total_samples']}")
print(f"HATE: {stats['hate_count']} | NON-HATE: {stats['non_hate_count']}")

In [ ]:
# Run evaluation on subset
samples = loader.get_samples(n=50, shuffle=True)

calculator = MetricsCalculator()
analyzer = FailureModeAnalyzer()

for sample in samples:
    result = await detector.detect(sample.image_path, "zero_shot")
    
    if not result.error:
        calculator.add_result(result.classification.label, sample.ground_truth_label)
        
        if result.classification.label != sample.ground_truth_label:
            analyzer.add_failure(result, sample.ground_truth_label)

# Print results
print(calculator.print_report())

In [ ]:
# Analyze failures
print(analyzer.generate_report())

## Custom Analysis

In [ ]:
# Use code-mixed prompt mode for Hindi-English memes
detector.set_vlm_prompt_mode("code_mixed")

result = await detector.detect("../data/samples/hinglish_meme.jpg", "cot")
display_result(result)

In [ ]:
# Reset to standard mode
detector.set_vlm_prompt_mode("standard")